This file should only be used to contain the code used to download data by using the selenium 

and

Upload data to minio

In [1]:
%pip install boto3
# Libraries maybe you need to install in your environment to use selenium and chromedriver below
# Note: Since selenium v4.6.0, the Selenium Manager is incorporated, which provides automated driver 
# management for Selenium, so there's no need for another chromedriver management library.
%pip install selenium==4.19.0

You should consider upgrading via the '/home/nguyenviet/.pyenv/versions/3.8.10/envs/icctm_env/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/home/nguyenviet/.pyenv/versions/3.8.10/envs/icctm_env/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import boto3
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [14]:
# The file download processing section should be placed here
# Please change the value of `download_directory` variable if you want to download and unzip in another folder
file_url = "https://drive.google.com/drive/folders/1EImvKyUQ_iJCfd2h-Dxjn_sqPyDV5tFC?usp=sharing"
download_directory = os.getcwd()

def wait_for_download(path: str, timeout: int = 10) -> None:
    """
    Wait for download to finish.
    If file in `path` argument is not found within timeout, a TimeoutError is raised.

    Args
    ----
    path : str
        Path to the intended file
    timeout : int (Optional, default 10)
        Number of seconds until timeout
    """

    elapsed_time = 0
    while elapsed_time < timeout:
        file_found = os.path.isfile(path)
        if file_found:
            return file_found
        time.sleep(1)
        elapsed_time += 1
    raise TimeoutError(
        f"File {path} not found, please check if your path is set correctly"
        " or your download operation is executed as intended."
    )


# Set default download directory option for the Chrome driver 
# Note: `download.default_directory` should have a slash '/' in the end in order to work properly
chrome_options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": download_directory+"/"
}
chrome_options.add_experimental_option("prefs", prefs)

# Initialize Chrome webdriver with `chrome_options`
driver = webdriver.Chrome(options=chrome_options)
wait = WebDriverWait(driver, 10)

try:
    # Access the url using the webdriver
    driver.get(file_url)
    # Get the file name to check later if the file is downloaded
    file_name = (
        WebDriverWait(driver, 10)
        .until(EC.presence_of_element_located((By.CLASS_NAME, "KL4NAf")))
        .text
    )
    # Trigger the download action
    element = (
        WebDriverWait(driver, 10)
        .until(EC.presence_of_element_located((By.CLASS_NAME, "akerZd")))
        .click()
    )
    # wait for the download to completed
    wait_for_download(f"{download_directory}/{file_name}")
finally:
    # Quit the webdriver whether the download action is success or not
    driver.quit()

In [15]:
# unzip the downloaded file
!unzip {download_directory}/{file_name} -d {download_directory}/

Archive:  /home/nguyenviet/Code/personal/icttm-assignment/icttm-test/data.zip
  inflating: /home/nguyenviet/Code/personal/icttm-assignment/icttm-test/data.json  
  inflating: /home/nguyenviet/Code/personal/icttm-assignment/icttm-test/__MACOSX/._data.json  
  inflating: /home/nguyenviet/Code/personal/icttm-assignment/icttm-test/data2.json  
  inflating: /home/nguyenviet/Code/personal/icttm-assignment/icttm-test/data3.json  


In [16]:
# Define S3 storage
# Please remember to set OBJ_STORAGE_ACCESS_KEY, OBJ_STORAGE_SECRET_KEY and OBJ_STORAGE_ENDPOINT environment variables prior to this step
obj_storage_access_key = os.getenv('OBJ_STORAGE_ACCESS_KEY', 'demo-access-key')
obj_storage_secret_key = os.getenv('OBJ_STORAGE_SECRET_KEY', 'demo-secret-key')
obj_storage_endpoint = os.getenv('OBJ_STORAGE_ENDPOINT', 'http://localhost:9000')

In [17]:
# Function to upload file to minio
def upload_file_to_minio(file_path, minio_bucket, minio_object_name):
    s3c = boto3.resource('s3',
                        endpoint_url=obj_storage_endpoint,
                        aws_access_key_id=obj_storage_access_key,
                        aws_secret_access_key=obj_storage_secret_key,
                        config=boto3.session.Config(signature_version='s3v4'),
                        verify=False
                        )
    s3c.Bucket(minio_bucket).upload_file(file_path, minio_object_name)

In [18]:
# Upload file to minio
upload_file_to_minio('data.json', 'data', 'data-raw/data.json')
upload_file_to_minio('data2.json', 'data', 'data-raw/data2.json')
upload_file_to_minio('data3.json', 'data', 'data-raw/data3.json')